# Acceleration Error

Test differences in step time in two segments that ought to start and end at the same time. 


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from math import log, sqrt
from itertools import count
import pandas as pd
import numpy as np
period = 4
import random
from trajectory.segments import SimSegment
from trajectory.segments import Joint, SegmentList

ModuleNotFoundError: No module named 'trajectory'

In [ ]:
ss1 = SimSegment(50, 20_000, 5000)
ss2 = SimSegment(50, 2_000, 500)
print(ss1.initial_params())
print(ss2.initial_params())
print ( (len(list(ss1.iter_discrete(period)))*period) - (len(list(ss2.iter_discrete(period)))*period) )


In [ ]:
ss2 = SimSegment(50, 2_000, 500)
list(ss2.iter_discrete(period))
ss2.sim_time

In [ ]:
def calc_segment(x, v0,  v1):
    t = abs(2. * float(x) / float(v1+v0) )
    a = (v1 - v0) / t
    return t

def simulate_segment(x, v0,  v1):
    period = 4
    ss = SimSegment(v0,v1,x)
    list(ss.iter_discrete(period))
    return ss.sim_time


def rand_segment():
    a_max = 1_000_000
    
    while True:
        x = random.randint(0,100000)
        v0 = random.randint(0,100000)
        v1 = random.randint(0,100000)

        t = abs(2. * float(x) / float(v1+v0) )
        a = (v1 - v0) / t
        
        if a <= a_max:
            return x, t, v0,  v1

x, t, v0,  v1 = rand_segment()
x, t, v0,  v1


In [ ]:
rs = [rand_segment() for _ in range(500)]

df = pd.DataFrame(rs, columns = 'x t v0 v1'.split())

df['sim_time'] = df.apply(lambda r : simulate_segment(r['x'], r.v0,  r.v1) , axis=1) 

df['err'] = (df.t - df.sim_time)/(df.t)
df['err'].hist()

In [ ]:
df.sort_values('err')

In [ ]:
simulate_segment(20000, 200,  40000), calc_segment(20000, 200,  40000)

In [ ]:
simulate_segment(20000, 100,  10000), calc_segment(20000, 100,  10000)

In [ ]:
from itertools import islice
ss = SimSegment(v0=0, v1=10000, x=100)
for e in islice(ss.iter_time(period),20):
    print(e)

In [ ]:
l = list(ss.iter_time(period))
l[-1], len(l)

In [ ]:
calc_segment(100, 0, 10000)

In [ ]:
a = 5
b = 7

( (1/a) - (1/b)), (b-a)/(a*b)

In [ ]:
sl = SegmentList([Joint(75000, 300_000, 300_000), Joint(75000, 300_000, 300_000)])
for i in range(2):
    sl.add_distance_segment([100_000, 10_000])

sl.update()
    
ss = []
for e in sl.sub_segments:
    print(e)
    ss.append([SimSegment(x=axis.x, v0=axis.v_i, v1=axis.v_f)  for i, axis in enumerate(e)]) 

In [ ]:
def run_ss(ss):
    l = list(ss.iter_time(period))
    return len(l), ss.sim_time, ss.a, ss.x, ss.v0, ss.v1

for i, e in enumerate(ss):
    print([run_ss(i) for i in e])